# Projeto de análise de dados sobre os registro de batalhas dos 1000 melhores jogadores em desafio 1v1 no clash royale, **Dado retirado em novembro de 2018**

### ⛏ Importação, tratamento, formatação básica dos dados e funções básicas

In [ ]:
# IMPORTANDO O PANDAS
import pandas as pd
from collections import Counter
from itertools import combinations
import os
import sys
# PARA QUE O NOTEBOOK CONSIGA ACESSAR A PASTA SRC
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
from src.features.criar_funcs import func_verificar_vencedor, contar_cartas_por_linhas
from src.utils.criar_funcs_utils import criar_dataframe_juntando_series

### TRATAMENTO E FORMATAÇÃO DE DADOS

In [ ]:
# IMPORTANDO A BASE DE DADOS
df_clash = pd.read_excel('../data/raw/Clash Royale Top Battles v1.xlsx')

In [ ]:
# FORMATANDO E TRATANDO O BÁSICO DO DATASET
del df_clash['Unnamed: 0']

In [ ]:
# CRIAR UMA CÓPIA DO DF_CLASH, PARA UTILIZAR SEM DANIFICAR O DF ORIGINAL
df_clash_copia = df_clash.copy()

In [ ]:
# CRIAR A COLUNA COM OS DECKS VENCEDORES, AINDA NULO
df_clash_copia['Deck Vencedor'] = None

In [ ]:
# FUNÇÃO PARA CRIAR A COLUNA, DIZENDO QUAL DECK FOI O VENCEDOR
def deck_vencedor(valor):
    if valor['p1_crowns'] > valor['p2_crowns']:
        df_clash_copia.loc[valor.name, 'Deck Vencedor'] = 1
    else:
        df_clash_copia.loc[valor.name, 'Deck Vencedor'] = 2

df_clash_copia.apply(deck_vencedor, axis=1)

### FUNÇÕES BÁSICAS E GENERALIZADAS

### 📍 Extrair o as cartas mais utilizadas nos decks

In [ ]:
# CRIA A LISTA COM AS CARTAS DOS 2 DECKS
lista_cartas_por_batalhalinha = df_clash.apply(contar_cartas_por_linhas, axis=1)

# CRIANDO UM Counter PARA CONTAR A QUANTIDADE DE APARIÇÕES DAS CARTAS EM TODAS AS LINHAS
dic_aparicoes_cartas = Counter()
for i, x in lista_cartas_por_batalhalinha.items():
   dic_aparicoes_cartas.update(x)

# CRIANDO UM DATAFRAME, MOSTRANDO A QUANTIDADE DE APARIÇÕES POR CARTA
df = pd.Series(dic_aparicoes_cartas)
df_cartas_mais_usadas = pd.DataFrame(df, columns=['Quantidade']).sort_values(by='Quantidade', ascending=False)

df_cartas_mais_usadas

### 📍 Extrair a taxa de vitória por carta (quantas vezes ela aparece nos decks vitoriosos)

In [ ]:
# GERA UMA LISTA COM OS DECKS VENCEDORES
lista_cartas_mais_vitoriosas = df_clash_copia.apply(func_verificar_vencedor, axis=1) # UMA SERIES ONDE CADA INDEX É UMA LISTA COM AS CARTAS DO DECK, OU SEJA O PRÓPIO DECK
# CRIANDO UM Counter PARA CONTAR A QUANTIDADE DE APARIÇÕES DAS CARTAS NOS DECKS VITORIOSOS
dic_aparicoes_cartas_mais_vitoriosas = Counter()
for i, x in lista_cartas_mais_vitoriosas.items():
    dic_aparicoes_cartas_mais_vitoriosas.update(x)

# # CRIANDO UM DATAFRAME, MOSTRANDO A QUANTIDADE DE APARIÇÕES POR CARTA NOS DECKS VENCEDORES
series_cartas_vitoriosas = pd.Series(dic_aparicoes_cartas_mais_vitoriosas)
df_cartas_mais_vitoriosas = pd.DataFrame(series_cartas_vitoriosas, columns=['Cartas mais Vencedoras']).sort_values(by='Cartas mais Vencedoras', ascending=False)
df_cartas_mais_vitoriosas

In [ ]:
# CRIAR UM DATAFRAME COM AS COLUNAS COM CARTAS QUE MAIS VENCERAM E SUAS APARIÇÕES
# df_taxa_vitorias = pd.concat([df_cartas_mais_usadas, df_cartas_mais_vitoriosas], axis=1)
df_taxa_vitorias = criar_dataframe_juntando_series(df_cartas_mais_usadas, df_cartas_mais_vitoriosas)

def tirar_taxa_vitoria(valor):
    return valor['Cartas mais Vencedoras'] / valor['Quantidade'] * 100

# CRIAR O DATAFRAME FORMATADO COM A TAXA DE VITÓRIA DE CADA CARTA
df_taxa_vitorias = pd.DataFrame(df_taxa_vitorias.apply(tirar_taxa_vitoria, axis=1),columns=['Taxa de Vitória'])
df_taxa_vitorias['Taxa de Vitória'] = df_taxa_vitorias['Taxa de Vitória'].map('{:.2f}'.format)
df_taxa_vitorias.sort_values(by='Taxa de Vitória',ascending=False, inplace=True)
df_taxa_vitorias

### 📍 Extrair combinações de cartas mais utilizadas

In [ ]:
# RETORNAR O DICIONARIO COM OS DECKS VITORIOSOS
dic_listas_combinacoes_vitoriosas = df_clash_copia.apply(func_verificar_vencedor, axis=1)
dic_listas_combinacoes_vitoriosas = dic_listas_combinacoes_vitoriosas.to_dict()

In [ ]:
# CRIAR O DATAFRAME, QUE CONTÉM OS VALORES DOS COMBOS MAIS VITORIOSAS
contagem_combinacoes_vitoriosas = Counter()
for i, deck in dic_listas_combinacoes_vitoriosas.items():
    comb = tuple(combinations(deck, 3))
    contagem_combinacoes_vitoriosas.update(comb)

series_combinacoes = pd.Series(contagem_combinacoes_vitoriosas).sort_values(ascending=False)
mask_formar_combos = series_combinacoes > 100
series_combinacoes  = series_combinacoes[mask_formar_combos]

df_qtd_combos_vencedores = pd.DataFrame(series_combinacoes, columns=['Combos mais Vencedores'])
df_qtd_combos_vencedores.index = df_qtd_combos_vencedores.index.map(lambda x: '+'.join(x))

In [ ]:
# RETORNA O DICIONÁRIO COM OS DECKS
dic_listas_combinacoes_mais_usadas = df_clash_copia.apply(contar_cartas_por_linhas, axis=1)
dic_listas_combinacoes_mais_usadas = dic_listas_combinacoes_mais_usadas.to_dict()

In [ ]:
# CRIAR O DATAFRAME, QUE CONTÉM OS COMBOS MAIS USADOS NAS PARTIDAS
contagem_combinacoes_mais_usadas = Counter()
for i, deck_m_usados in dic_listas_combinacoes_mais_usadas.items():
    comb_mais_usadas = list(combinations(deck_m_usados, 3))
    remover_duplicata = [tuple(sorted(c)) for c in comb_mais_usadas]
    contagem_combinacoes_mais_usadas.update(remover_duplicata)

series_combinacoes_mais_usadas = pd.Series(contagem_combinacoes_mais_usadas).sort_values(ascending=False)
mask_formar_combos_mais_usadas = series_combinacoes_mais_usadas > 100
series_combinacoes_mais_usadas = series_combinacoes_mais_usadas[mask_formar_combos_mais_usadas]

df_qtd_combos_mais_usados = pd.DataFrame(series_combinacoes_mais_usadas, columns=['Combos mais Utilizados'])
df_qtd_combos_mais_usados.index = df_qtd_combos_mais_usados.index.map(lambda x: '+'.join(x))
df_qtd_combos_mais_usados

In [ ]:
# CRIAR O DATAFRAME COM A TAXA DE VITÓRIA DOS COMBOS EM PARTIDAS

df_taxa_combos = criar_dataframe_juntando_series(df_qtd_combos_vencedores, df_qtd_combos_mais_usados)
df_taxa_combos.dropna(inplace=True)

def tirar_taxa_combos(valor):
    return valor['Combos mais Vencedores'] / valor['Combos mais Utilizados'] * 100

grupo = df_taxa_combos.apply(tirar_taxa_combos, axis=1)
df_taxa_combos = pd.DataFrame(grupo, columns=['Taxa de Vitória por Combos']).sort_values(by='Taxa de Vitória por Combos', ascending=False)
df_taxa_combos['Taxa de Vitória por Combos'] = df_taxa_combos['Taxa de Vitória por Combos'].map('{:.2f}'.format)
df_taxa_combos


### 📍 Extrair a média de coroas entre os vencedores e os pededores

In [ ]:
def tirar_media_coroas(valor):
    if valor['p1_crowns'] > valor['p2_crowns']:
        return valor['p1_crowns'] - valor['p2_crowns']
    else:
        return valor['p2_crowns'] - valor['p1_crowns']

grupo = df_clash_copia[['p1_crowns', 'p2_crowns']].apply(tirar_media_coroas, axis=1)
df_taxa_dif_coras = grupo.value_counts() / df_clash_copia.shape[0] * 100
df_taxa_dif_coras = pd.DataFrame(df_taxa_dif_coras)
df_taxa_dif_coras['count'] = df_taxa_dif_coras['count'].map('{:.2f}'.format)
df_taxa_dif_coras.rename(columns={
    'count': 'Taxa diferença coroas por Parida'
}, inplace=True
)

### 🛠 Gerar os arquivos formatados e processados

In [ ]:
df_clash_copia.to_pickle('../data/processed/df_clash_copia.pkl')
df_cartas_mais_usadas.to_pickle('../data/processed/df_cartas_mais_usadas.pkl')
df_taxa_vitorias.to_pickle('../data/processed/df_taxa_vitorias.pkl')
df_taxa_combos.to_pickle('../data/processed/df_taxa_combos.pkl')
df_taxa_dif_coras.to_pickle('../data/processed/df_taxa_dif_coroas.pkl')